# Estimación por razón
    
    - Se puede mejorar la estimación muestral de una variable cuando existe información censada, ya sea la misma variable en un periodo anterior u otra variable correlacionada.
    - La mejora en la estimación se consigue a través del coste del censo para obtener la variable adicional
    - Este método asume que la línea de regresión entre las dos variables pasa por el origen
    - Así mismo, el método es un caso específico de la estimación por regresión (en la cuál se puede relajar la restricción del origen y utilizar más de una variable

### 1] Generar una muestra sintética
    - Con parámetros y colinealidad arbitrarios

In [89]:
import pandas as pd
import numpy as np
from scipy.stats import t

# Settings
n = 30
N = 200
confidence = 0.95

# Generate population
index_name = ["ind "+str(i+1) for i in range(N)]
df_population = pd.DataFrame(index=index_name)
df_population["height"] = np.round(np.random.normal(loc=170, scale=8, size=N),2)
df_population["weight"] = np.round(df_population["height"] - np.random.uniform(low=95, high=100, size=N), 1)

# Calculate population data
r_pop = df_population["weight"].mean() / df_population["height"].mean()

# Select sample
df_sample = df_population.sample(n=n, replace=False)

### 2] Estimar la razón

    - La razón se define como la proporcionalidad o, de cierta forma, la correlación que existe entre la media de dos variables:

        R = y_mean / x_mean = sum(y) / sum(x)
        donde:
            x ex una variable censada para la cual se conoce N, s, mean
            y es una variable que ha sido sujeta a muestreo. En ocasiones y es x en un tiempo distinto

    - La varianza del estimador es:
        V(R) = (1-n/N) * 1/n * 1/x_mean^2 * 1/(n-1) * sum( (y_i - R*x_i)^2)
        donde:
            x_i es para la variable muestrada
        
    - Cuando n es grande se utiliza la función normal para definir los intervalos de confianza, mientras que cuando n es pequeña, se usa tStuden con n-2 grados de libertad (uno por cada variable)

In [90]:
# Height is considered as X, while weight as Y

# Estimate ratio
r_sample = df_sample["weight"].mean() / df_sample["height"].mean()
print("La razón entre peso y altura es de: "+str(round(r_sample,3)))

# Estimate confidence intervals
x_mean = df_sample["height"].mean()
r_var = (1-n/N) * 1/n * 1/x_mean**2 * 1/(n-1) * sum((df_sample["weight"] - r_sample*df_sample["height"])**2)
r_std = r_var**0.5
r_interval =  r_sample + r_std*t(df=n-1).ppf((0.5-confidence/2, 0.5+confidence/2))
print("La razón se estima un valor entre", np.round(r_interval,3))
print("La razón poblacional es: "+str(round(r_pop,3)))

La razón entre peso y altura es de: 0.42
La razón se estima un valor entre [0.411 0.43 ]
La razón poblacional es: 0.426


### 3] Mejorar la estimación de la media con la razón
    - Uno de las objetivos principales de la estimación por razón es disminuir la varianza en la estimación de los parámetros.
    - La estimación mejorada de la media es:
        mean = R * X_mean
        donde:
            R es el estimado en el paso anterior
            X_mean es la media en la variable censada (no la de la muestra)
            
    - La varianza del estimador mejorado es:
        V(mean) = (1-n/N) * 1/n * 1/(n-1) * sum( (y_i - R*x_i)^2 )
        donde:
            x_i es para la variable muestrada

In [91]:
# Height is considered as X, while weight as Y
print("Populatio\nmean: "+str(round(df_population["weight"].mean(),2)))

# Estimation usin SRS
mean = df_sample["weight"].mean()
mean_var = (1- n/N) * df_sample["weight"].var(ddof=1) / n
mean_std = mean_var**0.5
mean_interval =  mean + mean_std*t(df=n-1).ppf((0.5-confidence/2, 0.5+confidence/2))
print("\nSRS\nmean: "+str(round(mean,2))+" | mean_std: "+str(round(mean_std,2))+" | interval: "+str(np.round(mean_interval,2)))


# Improve estimation
mean = r_sample * df_population["height"].mean()
mean_var = (1-n/N) * 1/n * 1/(n-1) * sum((df_sample["weight"] - r_sample*df_sample["height"])**2)
mean_std = mean_var**0.5
mean_interval = mean + mean_std*t(df=n-1).ppf((0.5-confidence/2, 0.5+confidence/2))
print("\nRatio\nmean: "+str(round(mean,2))+" | mean_std: "+str(round(mean_std,2))+" | interval: "+str(np.round(mean_interval,2)))

Populatio
mean: 72.55

SRS
mean: 70.81 | mean_std: 1.33 | interval: [68.09 73.53]

Ratio
mean: 71.61 | mean_std: 0.77 | interval: [70.02 73.19]


### 4] Mejorar la estimación del TOTAL con la razón

    - La estimación mejorada del total es:
        total = R * X
            donde:
                R es el estimado de la razón sum(sum_yi) / sum(x_i)
                X es el total censado para la variable auxiliar
       
    - La varianza del estimador mejorado es:
            V(mean) = N^2 * (1-n/N) * 1/n * 1/(n-1) * sum( (y_i - R*x_i)^2 )
            donde:
                x_i es para la variable muestreada

In [92]:
# Height is considered as X, while weight as Y
print("Populatio\ntotal: "+str(round(df_population["weight"].sum(),2)))

# Estimation using SRS
total = N/n * df_sample["weight"].sum()
total_var = N**2 * (1-n/N) * df_sample["weight"].var(ddof=1)/n
total_std = total_var**0.5
total_interval = total + total_std*t(df=n-1).ppf((0.5-confidence/2, 0.5+confidence/2))
print("\nSRS\ntotal: "+str(round(total,2))+" | mean_std: "+str(round(total_std,2))+" | interval: "+str(np.round(total_interval,2)))

# Improve estimation
total = r_sample * df_population["height"].sum()
total_var = N**2 * (1-n/N) * 1/n * 1/(n-1) * sum((df_sample["weight"] - r_sample*df_sample["height"])**2)
total_std = total_var**0.5
total_interval = total + total_std*t(df=n-1).ppf((0.5-confidence/2, 0.5+confidence/2))
print("\nRatio\ntotal: "+str(round(total,2))+" | total_std: "+str(round(total_std,2))+" | interval: "+str(np.round(total_interval,2)))

Populatio
total: 14510.5

SRS
total: 14162.0 | mean_std: 265.61 | interval: [13618.77 14705.23]

Ratio
total: 14321.14 | total_std: 154.92 | interval: [14004.31 14637.98]
